# Chapter 16. Simultaneous Equations Models  
#### [Home](http://solomonegash.com/) | [Stata](http://solomonegash.com/woodridge1/index.html) | [R](http://solomonegash.com/econometrics/rbook1/index.html)

In [1]:

import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

from linearmodels.iv import IV2SLS

from wooldridge import *

### Example 16.5. Labor Supply of Married, Working Women

In [2]:
print(IV2SLS.from_formula('hours ~ 1 + [lwage ~ exper + expersq] + educ + age + kidslt6 + nwifeinc', 
                          data=dataWoo("mroz").dropna()).fit(cov_type='unadjusted'))

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  hours   R-squared:                     -2.0076
Estimator:                    IV-2SLS   Adj. R-squared:                -2.0433
No. Observations:                 428   F-statistic:                    17.450
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0037
Time:                        18:16:35   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      2225.7     570.52     3.9011     0.0001      1107.5      3343.9
educ          -183.75     58.684    -3.1312     0.00

In [3]:
df2=dataWoo("mroz").dropna(subset=['lwage'])

print(IV2SLS.from_formula(
    'lwage ~ 1 + educ + exper + expersq + [hours ~ age + kidslt6 + nwifeinc]', 
    data=df2).fit(cov_type='unadjusted'))

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1257
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1174
No. Observations:                 428   F-statistic:                    77.012
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0000
Time:                        18:16:35   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.6557     0.3358    -1.9527     0.0509     -1.3139      0.0024
educ           0.1103     0.0154     7.1488     0.00

### Example 16.6. Inflation and Openness 

In [4]:
df = dataWoo("openness")
print(smf.ols('open ~ lpcinc + lland', data = df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   open   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     45.17
Date:                Tue, 02 Jul 2024   Prob (F-statistic):           4.45e-15
Time:                        18:16:36   Log-Likelihood:                -488.44
No. Observations:                 114   AIC:                             982.9
Df Residuals:                     111   BIC:                             991.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    117.0845     15.848      7.388      0.0

In [5]:
print(IV2SLS.from_formula('inf ~ [open ~ lland] + lpcinc + 1', data = df).fit())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                    inf   R-squared:                      0.0309
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0134
No. Observations:                 114   F-statistic:                    5.1930
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0745
Time:                        18:16:36   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      26.899     10.775     2.4964     0.0125      5.7802      48.018
lpcinc         0.3758     1.3603     0.2763     0.78

### Example 16.7. Testing the Permanent Income Hypothesis 

In [6]:
df = dataWoo("consump").dropna()
pi_reg = IV2SLS.from_formula('gc ~ 1 + [gy + r3 ~ gy_1 + gc_1 + r3_1]', data = df).fit()
print(pi_reg)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gc   R-squared:                      0.6767
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6558
No. Observations:                  34   F-statistic:                    17.903
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0001
Time:                        18:16:36   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0082     0.0035     2.3826     0.0172      0.0015      0.0150
gy             0.5821     0.1376     4.2295     0.00

In [7]:
df['uhat'] = pi_reg.resids
print(smf.ols('uhat ~ uhat.shift(1)', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   uhat   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                    0.3062
Date:                Tue, 02 Jul 2024   Prob (F-statistic):              0.584
Time:                        18:16:36   Log-Likelihood:                 115.55
No. Observations:                  33   AIC:                            -227.1
Df Residuals:                      31   BIC:                            -224.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6.367e-05      0.001      0.049

In [8]:
df['uhat2']  = df['uhat'].shift(1)
df2=df.dropna()
print(IV2SLS.from_formula('gc ~ 1 + [ gy + r3  + uhat2 ~  gy_1 + gc_1 + r3_1]', data = df2).fit())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     gc   R-squared:                      0.4725
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4180
No. Observations:                  33   F-statistic:                    11.117
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0111
Time:                        18:16:36   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept  -6.569e-05     0.0073    -0.0090     0.9929     -0.0144      0.0143
gy             0.9765     0.3316     2.9446     0.00

### Example 16.8. Effect of Prison Population on Violent Crime Rates

In [9]:
df = dataWoo("prison").dropna()
print(IV2SLS.from_formula('gcriv ~ 1 + [gpris ~ final1 + final2] + gpolpc + gincpc + cunem + cblack + cmetro + cag0_14 + cag15_17 + cag18_24 + cag25_34', data=df).fit(cov_type='unadjusted'))
print(smf.ols('gcriv ~ gpris + gpolpc + gincpc + cunem + cblack + cmetro + cag0_14 + cag15_17 + cag18_24 + cag25_34', data=df).fit().summary())

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  gcriv   R-squared:                     -0.2447
Estimator:                    IV-2SLS   Adj. R-squared:                -0.2624
No. Observations:                 714   F-statistic:                    59.380
Date:                Tue, Jul 02 2024   P-value (F-stat)                0.0000
Time:                        18:16:36   Distribution:                 chi2(10)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0363     0.0242     1.5039     0.1326     -0.0110      0.0837
gpolpc         0.0735     0.0690     1.0650     0.28